In [35]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import cPickle as pickle

In [2]:
%%time
train_df = pd.read_csv('train.csv')
artist_df = pd.read_csv('artists.csv')
profile_df = pd.read_csv('profiles.csv')

Wall time: 16.9 s


In [3]:
profile_df.set_index('user', inplace=True)

In [4]:
print profile_df.shape
profile_df.head()

(233286, 3)


,sex,age,country
user,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25,Sweden
5909125332c108365a26ccf0ee62636eee08215c,m,29,Iceland
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30,United States
63268cce0d68127729890c1691f62d5be5abd87c,m,21,Germany
02871cd952d607ba69b64e2e107773012c708113,m,24,Netherlands


In [5]:
%%time
for k in range(10):
    profile_df[str(10*k) + 's'] = ((profile_df.age >= 10*k) & (profile_df.age < 10*k+10)).astype(int)
print profile_df.shape

(233286, 13)
Wall time: 197 ms


In [6]:
%%time
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.sex)], axis=1)
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.age)], axis=1)
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.country)], axis=1)

Wall time: 3.3 s


In [7]:
del profile_df['sex']
del profile_df['age']
del profile_df['country']

In [8]:
print profile_df.shape
profile_df.head()

(233286, 364)


,0s,10s,20s,30s,40s,50s,60s,70s,80s,90s,...,Vanuatu,Venezuela,Viet Nam,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe
user,,,,,,,,,,,,,,,,,,,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5909125332c108365a26ccf0ee62636eee08215c,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63268cce0d68127729890c1691f62d5be5abd87c,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02871cd952d607ba69b64e2e107773012c708113,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
m = sparse.lil_matrix(profile_df.values)
m

<233286x364 sparse matrix of type '<type 'numpy.float64'>'
	with 823169 stored elements in LInked List format>

In [15]:
cols = list(profile_df.columns)
artist_list = [tup[1] for tup in artist_df.itertuples()]

cols.extend(artist_list)
m = sparse.hstack((m, sparse.csr_matrix((m.shape[0], len(artist_list)))), format='lil')
m

<233286x2364 sparse matrix of type '<type 'numpy.float64'>'
	with 823169 stored elements in LInked List format>

In [16]:
col_dict = {col:i for i,col in enumerate(cols)}

In [17]:
row_dict = {row:i for i, row in enumerate(profile_df.index)}

In [29]:
%%time
for row in train_df.itertuples():
    m[row_dict[row[1]], col_dict[row[2]]] = row[3]

Wall time: 1min 5s


In [32]:
m = m.tocsr()

In [36]:
%%time
with open('data.p', 'w') as p:
    pickle.dump((row_dict, col_dict, m), p)

Wall time: 24.7 s


###Clustering

In [76]:
from sklearn.decomposition import PCA, SparsePCA, MiniBatchSparsePCA
from sklearn.preprocessing import MaxAbsScaler

In [104]:
scaler = MaxAbsScaler()
m = scaler.fit_transform(m)

In [105]:
pca = MiniBatchSparsePCA(n_components=10)

In [112]:
%%time
pca.fit(m[0:100000,:].toarray())

Wall time: 21.3 s


MiniBatchSparsePCA(alpha=1, batch_size=3, callback=None, method='lars',
          n_components=10, n_iter=100, n_jobs=1, random_state=None,
          ridge_alpha=0.01, shuffle=True, verbose=False)

In [150]:
print map(lambda k : sum(pca.components_[k] != 0), range(10))
print np.array(cols)[pca.components_[9]!=0]

[0, 1, 0, 0, 2, 0, 0, 0, 1, 21]
['20s' 'f' 'm' '20.0' 'Australia' 'Brazil' 'Canada' 'Finland' 'France'
 'Germany' 'Italy' 'Mexico' 'Netherlands' 'Norway' 'Poland'
 'Russian Federation' 'Spain' 'Sweden' 'Turkey' 'United Kingdom'
 'United States']
